In [ ]:
!pip install pymysql
import pymysql
import numpy as np

# Define user input mapping function
def map_user_raw_input(user_data_original):
    mapped_data = user_data_original.copy()
    # Tobacco mapping
    tobacco_amount = mapped_data.get('tobacco')
    try:
        mapped_data['tobacco'] = int(tobacco_amount) if tobacco_amount is not None and tobacco_amount != '' else np.nan
    except (ValueError, TypeError):
        mapped_data['tobacco'] = np.nan
    # BE3_31 mapping
    walk_freq_map = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 99: np.nan}
    mapped_data['BE3_31'] = walk_freq_map.get(mapped_data.get('BE3_31'), mapped_data.get('BE3_31'))
    # BD1_11 mapping (weekly frequency)
    alcohol_freq_map = {1: 0, 2: 0.16, 3: 0.23, 4: 0.80, 5: 2.5, 6: 4, 8: 0, 9: np.nan}
    mapped_data['BD1_11'] = alcohol_freq_map.get(mapped_data.get('BD1_11'), mapped_data.get('BD1_11'))
    # L_BR_FQ mapping
    breakfast_freq_map = {1: 6.0, 2: 3.5, 3: 1.5, 4: 0.0, 9: np.nan}
    mapped_data['L_BR_FQ'] = breakfast_freq_map.get(mapped_data.get('L_BR_FQ'), mapped_data.get('L_BR_FQ'))
    return mapped_data




# 사용자 입력값 (예시)
user_data = {
    "member_id": "MEM001",
    "checkup_id": "CHK20240525",
    "흡연": 4,
    "음주": 3, 
    "아침식사": 2,
    "걷기운동": 1
}

mapped_data = map_user_raw_input(user_data)



# 아이콘 상태 결정 함수
def get_icon_status(mapped_data):
    return {
        "smoking_icon": "on" if mapped_data["흡연"] >= 1 else "off", # 흡연이 1 이상이면 on, 아니면 off
        "drinking_icon": "on" if mapped_data["음주"] >= 2 else "off", # 음주가 2 이상이면 on, 아니면 off
        "breakfast_icon": "on" if mapped_data["아침식사"] <= 2 else "off", # 아침식사가 2 이하이면 on, 아니면 off
        "walking_icon": "on" if mapped_data["걷기운동"] <= 2 else "off" # 걷기운동이 2 이하이면 on, 아니면 off
    }

# DB 연결
conn = pymysql.connect(
    host='health-db.c1cwqmcao40y.ap-northeast-2.rds.amazonaws.com',
    user='admin',
    password='HealthCheckup123!',
    db='health',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

icon_status = get_icon_status(mapped_data)

# INSERT 실행
try:
    with conn.cursor() as cursor:
        sql = """
        INSERT INTO icon_status (
            member_id, checkup_id, smoking_icon, drinking_icon, breakfast_icon, walking_icon
        ) VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (
            mapped_data["member_id"],
            mapped_data["checkup_id"],
            icon_status["smoking_icon"],
            icon_status["drinking_icon"],
            icon_status["breakfast_icon"],
            icon_status["walking_icon"]
        ))
    conn.commit()
    print("데이터 저장 완료")
finally:
    conn.close()


Defaulting to user installation because normal site-packages is not writeable
데이터 저장 완료


DEPRECATION: Loading egg at c:\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import pymysql
import numpy as np

# Define user input mapping function
def map_user_raw_input(user_data_original):
    mapped_data = user_data_original.copy()

    # Tobacco mapping
    tobacco_amount = mapped_data.get('tobacco')
    try:
        # Converts to int; handles None, empty string, or non-int by setting to np.nan
        mapped_data['tobacco'] = int(tobacco_amount) if tobacco_amount is not None and tobacco_amount != '' else np.nan
    except (ValueError, TypeError):
        mapped_data['tobacco'] = np.nan

    # Walking frequency mapping (BE3_31 in original survey)
    # Maps categorical input to numerical days/frequency
    walk_freq_map = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 99: np.nan}
    mapped_data['walking_freq'] = walk_freq_map.get(mapped_data.get('walking_freq'), np.nan) # Using np.nan as default if key not found

    # Alcohol frequency mapping (BD1_11 in original survey)
    # 1주일간 음주 빈도 
    alcohol_freq_map = {1: 0, 2: 0.16, 3: 0.23, 4: 0.80, 5: 2.5, 6: 4, 8: 0, 9: np.nan}
    mapped_data['alcohol_freq'] = alcohol_freq_map.get(mapped_data.get('alcohol_freq'), np.nan) # Using np.nan as default

    # Breakfast frequency mapping (L_BR_FQ in original survey)
    # Maps categorical input to estimated weekly frequency
    breakfast_freq_map = {1: 6.0, 2: 3.5, 3: 1.5, 4: 0.0, 9: np.nan}
    mapped_data['breakfast_freq'] = breakfast_freq_map.get(mapped_data.get('breakfast_freq'), np.nan) # Using np.nan as default

    return mapped_data

# Icon status determination function
def get_icon_status(mapped_data):
    # Retrieve mapped values, safely handling cases where keys might be missing or values are NaN
    smoking_val = mapped_data.get('tobacco')
    drinking_val = mapped_data.get('alcohol_freq')
    breakfast_val = mapped_data.get('breakfast_freq')
    walking_val = mapped_data.get('walking_freq')

    return {
        # Smoking: 'on' if smoking amount is 1 or more, otherwise 'off'
        "smoking_icon": "on" if smoking_val is not None and not np.isnan(smoking_val) and smoking_val >= 1 else "off",
        # Drinking: 'on' if alcohol frequency is 2 or more (e.g., 2.5 times/week for category 5), otherwise 'off'
        "drinking_icon": "on" if drinking_val is not None and not np.isnan(drinking_val) and drinking_val >= 2 else "off",
        # Breakfast: 'on' if breakfast frequency is 2 or less (e.g., 3.5 times/week for category 2), otherwise 'off'
        "breakfast_icon": "on" if breakfast_val is not None and not np.isnan(breakfast_val) and breakfast_val <= 2 else "off",
        # Walking: 'on' if walking frequency is 2 or less (e.g., 2 days/week for category 3), otherwise 'off'
        "walking_icon": "on" if walking_val is not None and not np.isnan(walking_val) and walking_val <= 2 else "off"
    }

# --- Example Usage ---

# User input values (using English variable names as requested)
user_data = {
    "member_id": "MEM001",
    "checkup_id": "CHK20240525",
    "tobacco": 4,           # Tobacco amount (e.g., number of cigarettes)
    "alcohol_freq": 3,      # Alcohol frequency (e.g., 3: 2-3 times/month)
    "breakfast_freq": 2,    # Breakfast frequency (e.g., 2: 5-6 times/week)
    "walking_freq": 1       # Walking frequency (e.g., 1: Never)
}

# Map the raw user input to standardized values
mapped_data = map_user_raw_input(user_data)

# Determine the icon status based on mapped data
icon_status = get_icon_status(mapped_data)

print(f"Mapped Data: {mapped_data}")
print(f"Icon Status: {icon_status}")

# --- Database Connection and Insertion ---

# DB connection parameters
db_host = 'health-db.c1cwqmcao40y.ap-northeast-2.rds.amazonaws.com'
db_user = 'admin'
db_password = 'HealthCheckup123!'
db_name = 'health'

conn = None # Initialize conn to None
try:
    # Connect to the database
    conn = pymysql.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

    with conn.cursor() as cursor:
        # SQL INSERT statement
        sql = """
        INSERT INTO icon_status (
            member_id, checkup_id, smoking_icon, drinking_icon, breakfast_icon, walking_icon
        ) VALUES (%s, %s, %s, %s, %s, %s)
        """
        # Execute the INSERT statement with mapped data and icon statuses
        cursor.execute(sql, (
            user_data["member_id"], # Using original member_id from user_data
            user_data["checkup_id"], # Using original checkup_id from user_data
            icon_status["smoking_icon"],
            icon_status["drinking_icon"],
            icon_status["breakfast_icon"],
            icon_status["walking_icon"]
        ))
    
    # Commit the transaction to save changes to the database
    conn.commit()
    print("데이터 저장 완료 (Data saved successfully)")

except pymysql.Error as e:
    # Handle database errors
    print(f"데이터베이스 오류 발생: {e}")
    if conn:
        conn.rollback() # Rollback changes if an error occurs
except Exception as e:
    # Handle any other unexpected errors
    print(f"예상치 못한 오류 발생: {e}")
finally:
    # Ensure the database connection is closed
    if conn:
        conn.close()
        print("데이터베이스 연결 종료 (Database connection closed)")

Mapped Data: {'member_id': 'MEM001', 'checkup_id': 'CHK20240525', 'tobacco': 4, 'alcohol_freq': 0.23, 'breakfast_freq': 3.5, 'walking_freq': 0}
Icon Status: {'smoking_icon': 'on', 'drinking_icon': 'off', 'breakfast_icon': 'off', 'walking_icon': 'on'}
데이터 저장 완료 (Data saved successfully)
데이터베이스 연결 종료 (Database connection closed)
